In [1]:
import pdfplumber
import pandas as pd
from src.widgets_helpers import file_selector_from_filder
import re
from src.parse_pdf import parse_pdf_text, separar_columnas_cc, extract_text_from_pdf_first_page, convert_transactions_to_df, convert_transactions_tarjeta_to_df, extract_text_from_pdf_rest_of_pages, separar_columnas_tajeta

In [21]:
file = file_selector_from_filder("extractos/")

Dropdown(description='Archivos:', layout=Layout(width='50%'), options=(('Extracto_Febrero_2024.pdf', 'extracto…

In [22]:
file

'extractos/Extracto_Febrero_2024.pdf'

In [16]:
text_cc = extract_text_from_pdf_first_page(file)

In [17]:
transacciones_cc = parse_pdf_text(text_cc,separar_columnas_cc)

In [18]:
df_cuenta = convert_transactions_to_df(transacciones_cc)

In [19]:
df_cuenta

,Fecha,Ref,Fecha Valor,Descripción,cargo,abono,Saldo
0,2024-02-02,846,2024-02-01,RECIBO PLATINUM,"1.285,52",None,"5.793,33"
1,2024-02-02,006,2024-02-01,PAGO BIZUM A NATALIA;GRANDA;AL,"4,40",None,"5.788,93"
2,2024-02-07,134,2024-02-07,PAGO BIZUM A ROBERTO;PERINELLI,"85,00",None,"5.703,93"
3,2024-02-10,201,2024-02-10,PAGO BIZUM A ANE;RAMSDEN;ARRIB,"33,00",None,"5.670,93"
4,2024-02-10,322,2024-02-10,PAGO BIZUM A SAMIR RUBEN;AUYAN,"70,80",None,"5.600,13"
5,2024-02-12,042,2024-02-12,PAGO BIZUM A DANIEL;TANO;LEAL,"5,00",None,"5.595,13"
6,2024-02-12,505,2024-02-11,PAGO BIZUM A ANE;RAMSDEN;ARRIB,"5,00",None,"5.590,13"
7,2024-02-12,662,2024-02-11,PAGO BIZUM A NATALIA;GRANDA;AL,"31,50",None,"5.558,63"
8,2024-02-13,980,2024-02-13,RECIBO /SIMYO,"23,43",None,"5.535,20"
9,2024-02-13,014,2024-02-11,CAJERO LA CAIXA OF.6738,"80,00",None,"5.455,20"


In [7]:
text_t = extract_text_from_pdf_rest_of_pages(file)

In [8]:
transacciones_t = parse_pdf_text(text_t, separar_columnas_tajeta) 

In [9]:
df_tarjeta = convert_transactions_tarjeta_to_df(transacciones_t)

In [10]:

df_tarjeta

,Fecha,Descripción,cargo,abono
0,2024-01-31,"CRUNCH IB RICO, LAS PALMAS","2,50",None
1,2024-01-31,"HD PORTUGAL, 35010","6,57",None
2,2024-01-31,"COM. USO REDES INTERNACION., SAN 0,60","0,56",None
3,2024-01-31,"CHATGPT SUBSCRIPTION, SAN FRANCI 20,00","18,51",None
4,2024-02-01,"PAYPAL *GLOVO, 35314369001","29,79",None
...,...,...,...,...
118,2024-02-27,"COM. USO REDES INTERNACION., 402-935-7733","0,14",None
119,2024-02-27,"PAYPAL *MEDIUM.COM, 402-935-7733","4,82",None
120,2024-02-28,"EL MOMENTO, LAS PALMAS","3,60",None
121,2024-02-04,"RESTAURANTE LAS NASAS, AGAETE","24,88",None
